In [1]:
# Add this at the start of your notebook
import os
from pathlib import Path
import PyQt6

# Set Qt plugin path explicitly
qt_path = Path(PyQt6.__file__).parent / "Qt6" / "plugins" / "platforms"
if qt_path.exists():
    os.environ['QT_QPA_PLATFORM_PLUGIN_PATH'] = str(qt_path)


In [2]:
from tmseegpy.ica_topo_classifier import ICATopographyClassifier
import mne


Using matplotlib as 2D backend.
Using qt as 2D backend.
Cannot switch Qt versions for this session; you must use qt6.
Set QT_QPA_PLATFORM_PLUGIN_PATH to: /Users/alexe/anaconda3/envs/tmseeg-env/lib/python3.11/site-packages/PyQt6/Qt6/plugins/platforms


In [3]:
fname = '/Users/alexe/Kaggle/get_done/Magnus_test/out/2024-12-04T163850_wake_phase_1/preprocessing_steps/2_epochs_bad_no_epoch/2024-12-04T163850_wake_phase_1_2_epochs_bad_no_epoch_epo.fif'

epochs = mne.read_epochs(fname)

Reading /Users/alexe/Kaggle/get_done/Magnus_test/out/2024-12-04T163850_wake_phase_1/preprocessing_steps/2_epochs_bad_no_epoch/2024-12-04T163850_wake_phase_1_2_epochs_bad_no_epoch_epo.fif ...
    Found the data of interest:
        t =    -800.00 ...     800.00 ms
        0 CTF compensation matrices available
Not setting metadata
95 matching events found
No baseline correction applied
0 projection items activated


In [ ]:
epochs.plot(scalings='auto')

In [4]:
ica = mne.preprocessing.ICA(
    max_iter="auto",
    method="fastica",
    random_state=42
)

ica.fit(epochs)


Fitting ICA to data using 32 channels (please be patient, this may take a while)


/var/folders/9m/04qxqpfn46d0n6sjf6pkwwh80000gn/T/ipykernel_39229/1770999120.py:7: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(epochs)


Selecting by non-zero PCA components: 29 components
Fitting ICA took 18.4s.


Method,fastica
Fit parameters,algorithm=parallelfun=logcoshfun_args=Nonemax_iter=1000
Fit,58 iterations on epochs (760095 samples)
ICA components,29
Available PCA components,32
Channel types,eeg
ICA components marked for exclusion,—


In [6]:
ica.plot_sources(epochs)

Not setting metadata
95 matching events found
No baseline correction applied
0 projection items activated
Using pyopengl with version 3.1.7


In [5]:
classifier = ICATopographyClassifier(ica, epochs)
results = classifier.classify_all_components()

# Print detailed results
for idx, res in results.items():
    print(f"\nComponent {idx}:")
    print(f"Classification: {res['classification']}")
    print(f"Reasons: {res['details']['reasons']}")
    print(f"Max Z-score: {res['details']['max_zscore']:.2f}")
    print(f"Number of peaks: {res['details']['n_peaks']}")


Component 0:
Classification: non_artifact
Reasons: ['focal_pattern (peaks=0)']
Max Z-score: 2.51
Number of peaks: 0

Component 1:
Classification: artifact
Reasons: ['high_amplitude (z=4.26)', 'focal_pattern (peaks=1)', 'concentrated_activity']
Max Z-score: 4.26
Number of peaks: 1

Component 2:
Classification: non_artifact
Reasons: ['focal_pattern (peaks=0)']
Max Z-score: 2.32
Number of peaks: 0

Component 3:
Classification: non_artifact
Reasons: ['focal_pattern (peaks=0)']
Max Z-score: 3.03
Number of peaks: 0

Component 4:
Classification: non_artifact
Reasons: ['focal_pattern (peaks=0)']
Max Z-score: 3.18
Number of peaks: 0

Component 5:
Classification: non_artifact
Reasons: ['focal_pattern (peaks=0)']
Max Z-score: 2.86
Number of peaks: 0

Component 6:
Classification: artifact
Reasons: ['high_amplitude (z=3.59)', 'focal_pattern (peaks=1)', 'concentrated_activity']
Max Z-score: 3.59
Number of peaks: 1

Component 7:
Classification: non_artifact
Reasons: ['focal_pattern (peaks=0)']
Max Z

In [ ]:
import numpy as np

artifact_components = [0, 1, 3, 6, 8, 11, 12, 13, 14, 15, 16, 20, 24, 28]
non_artifact_components = [i for i in range(29) if i not in artifact_components]

# Get patterns for both groups
def analyze_component_patterns(classifier, artifact_indices, non_artifact_indices):
    artifact_patterns = [classifier.patterns[:, i] for i in artifact_indices]
    non_artifact_patterns = [classifier.patterns[:, i] for i in non_artifact_indices]

    # Get z-scored patterns
    artifact_z = [classifier._normalize_pattern(p) for p in artifact_patterns]
    non_artifact_z = [classifier._normalize_pattern(p) for p in non_artifact_patterns]

    # Analyze key features
    def get_pattern_features(pattern):
        z_pattern = classifier._normalize_pattern(pattern)
        max_abs = np.max(np.abs(z_pattern))
        peak_indices = np.where(np.abs(z_pattern) > classifier.focal_intensity_threshold)[0]
        peak_positions = [classifier.pos[idx] for idx in peak_indices]

        # Calculate distances from center for peaks
        distances = [np.linalg.norm(pos) for pos in peak_positions]

        return {
            'max_abs_zscore': max_abs,
            'n_peaks': len(peak_indices),
            'peak_distances': distances,
            'pattern': z_pattern
        }

    artifact_features = [get_pattern_features(p) for p in artifact_patterns]
    non_artifact_features = [get_pattern_features(p) for p in non_artifact_patterns]

    return artifact_features, non_artifact_features

# Run analysis
artifact_features, non_artifact_features = analyze_component_patterns(
    classifier, artifact_components, non_artifact_components)

# Print summary statistics
def print_feature_stats(features, name):
    max_zscores = [f['max_abs_zscore'] for f in features]
    n_peaks = [f['n_peaks'] for f in features]
    distances = [d for f in features for d in f['peak_distances']]

    print(f"\n{name} Components Statistics:")
    print(f"Max Z-scores: mean={np.mean(max_zscores):.2f}, std={np.std(max_zscores):.2f}")
    print(f"Number of peaks: mean={np.mean(n_peaks):.2f}, std={np.std(n_peaks):.2f}")
    print(f"Peak distances from center: mean={np.mean(distances):.2f}, std={np.std(distances):.2f}")

print_feature_stats(artifact_features, "Artifact")
print_feature_stats(non_artifact_features, "Non-Artifact")